In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [3]:
df=pd.read_csv('gurgaon_cleaned.csv')

In [4]:
df.head(5)

,society,price,area_persqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,sector,area,type
0,maa bhagwati residency,0.45,5000.0,Carpet area: 900 (83.61 sq.m.),2.0,2.0,1,not avilable,4.0,West,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...",So with lift.Maa bhagwati residency is one of ...,"['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...",sector 7,900.0sq-ft,flat
1,apna enclave,0.5,7692.0,Carpet area: 650 (60.39 sq.m.),2.0,2.0,1,not avilable,1.0,West,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","Property situated on main road, railway statio...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...",sector 3,650.026sq-ft,flat
2,tulsiani easy in homes,0.4,6722.0,Carpet area: 595 (55.28 sq.m.),2.0,2.0,3,not avilable,12.0,NaN,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...","This property is 15 km away from badshapur, gu...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian...",sohna road,595.061sq-ft,flat
3,smart world orchard,1.47,12250.0,Carpet area: 1200 (111.48 sq.m.),2.0,2.0,2,Study Room,2.0,NaN,2023-12-01 00:00:00,"['Sector 55-56 Metro station', 'Bestech Centra...",Near to metro station of sector 56 and opposit...,NaN,"['Security / Fire Alarm', 'Private Garden / Te...",sector 61,1200.0sq-ft,flat
4,parkwood westend,0.7,5204.0,Super Built up area 1345(124.95 sq.m.),2.0,2.0,3,Study Room,5.0,NaN,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",We are the proud owners of this 2 bhk alongwit...,[],NaN,sector 92,1345.119sq-ft,flat


In [5]:
df['areaWithType']=df['areaWithType'].astype(str)

In [6]:
import re
def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)# sq-ft extract 
    if match:
        return float(match.group(1))
    return None



In [7]:
# This function extracts the Built Up area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text) # extract sqft value
    if match:
        return float(match.group(1))
    return None

In [8]:
'''2262(210.15 sq.m.)/ 1580 sq.ft. (146.79 sq.m.)expected format in search method 
   we pass the value in area_value the number followed by ('''
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{}\s*(sq\.ft\. )?\((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(2))# inside braces all are sq-m value
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value

In [9]:
# Extract Super Built up area and convert to sqft if needed
df['super_built_up_area'] = df['areaWithType'].astype(str).apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)

In [10]:
df['areaWithType'].value_counts()

areaWithType
nan                                                                                                                             92
Plot area 360(301.01 sq.m.)                                                                                                     39
Plot area 300(250.84 sq.m.)                                                                                                     27
Plot area 200(167.23 sq.m.)                                                                                                     21
Plot area 270(225.75 sq.m.)                                                                                                     19
Plot area 502(419.74 sq.m.)                                                                                                     19
Super Built up area 1578(146.6 sq.m.)                                                                                           17
Super Built up area 1950(181.16 sq.m.)Carpet area: 1161 sq.ft. (107.86

In [11]:
df.columns

Index(['society', 'price', 'area_persqft', 'areaWithType', 'bedRoom',
       'bathroom', 'balcony', 'additionalRoom', 'floorNum', 'facing',
       'agePossession', 'nearbyLocations', 'description', 'furnishDetails',
       'features', 'sector', 'area', 'type', 'super_built_up_area',
       'built_up_area', 'carpet_area'],
      dtype='object')

In [12]:
all_nan=df[(df['super_built_up_area'].isnull())&(df['built_up_area'].isnull())&(df['carpet_area'].isnull())]

In [13]:
# extract plot area and put into bulit_uparea
def get_plotarea(text):
    match=re.search(r'Plot area (\d*\.?\d+)',text)
    if match:
        return float(match.group(1))
    else:
        return np.nan

In [14]:
all_nan['built_up_area']=all_nan['areaWithType'].astype(str).apply(get_plotarea)

/var/folders/9h/szcjr1mj4c51q1_y7hpk6bvm0000gn/T/ipykernel_27283/506463397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_nan['built_up_area']=all_nan['areaWithType'].astype(str).apply(get_plotarea)


In [15]:
all_nan

,society,price,area_persqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,sector,area,type,super_built_up_area,built_up_area,carpet_area
2846,signature global solera,NaN,NaN,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 107,nansq-ft,flat,NaN,NaN,NaN
2847,tulip monsella,NaN,33198.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 53,nansq-ft,flat,NaN,NaN,NaN
2848,my home,NaN,4400.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new sector 2,nansq-ft,flat,NaN,NaN,NaN
2849,breez global hill view,NaN,5470.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sohna road,nansq-ft,flat,NaN,NaN,NaN
2918,pareena coban residences,NaN,5759.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 99a,nansq-ft,flat,NaN,NaN,NaN
2919,ikon tower baani city centre,NaN,12437.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,golf course ext road,nansq-ft,flat,NaN,NaN,NaN
2920,greenopolis,NaN,4820.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 89,nansq-ft,flat,NaN,NaN,NaN
2922,maple heights,NaN,8888.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 43,nansq-ft,flat,NaN,NaN,NaN
2923,assotech blith,NaN,6593.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 99,nansq-ft,flat,NaN,NaN,NaN
2988,chanderlok society,NaN,7234.0,nan,NaN,NaN,NaN,not avilable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sector 28,nansq-ft,flat,NaN,NaN,NaN


In [16]:
all_nan['area'].value_counts()

area
(301 sq.m.) Plot Area            39
(84 sq.m.) Plot Area             33
(251 sq.m.) Plot Area            29
(167 sq.m.) Plot Area            25
(226 sq.m.) Plot Area            20
(420 sq.m.) Plot Area            20
nansq-ft                         19
(418 sq.m.) Plot Area            18
(125 sq.m.) Plot Area            17
(180 sq.m.) Plot Area            13
(135 sq.m.) Plot Area            12
(209 sq.m.) Plot Area            12
(334 sq.m.) Plot Area            11
(50 sq.m.) Plot Area             11
(42 sq.m.) Plot Area             11
(201 sq.m.) Plot Area            10
(264 sq.m.) Plot Area            10
(33 sq.m.) Plot Area             10
(220 sq.m.) Plot Area             9
(171 sq.m.) Plot Area             9
(145 sq.m.) Plot Area             8
(502 sq.m.) Plot Area             8
(293 sq.m.) Plot Area             7
(134 sq.m.) Plot Area             7
(242 sq.m.) Plot Area             6
(372 sq.m.) Plot Area             6
(161 sq.m.) Plot Area             6
(51 sq.m.) Plot Area   

In [17]:
all_nan.loc[all_nan['area'] == '(84 sq.m.) Carpet Area', 'carpet_area'] = 904.16

In [18]:
all_nan.loc[all_nan['area'] == '(49 sq.m.) Carpet Area'] = 527.432

In [19]:
all_nan.loc[all_nan['area'] == '(84 sq.m.) Carpet Area', 'area'] = np.nan
all_nan.loc[all_nan['area'] == '(49 sq.m.) Carpet Area', 'area'] = np.nan
all_nan.loc[all_nan['area'] == 'nansq-ft', 'area'] = np.nan

In [20]:
all_nan[['area','built_up_area']]

,area,built_up_area
2846,NaN,NaN
2847,NaN,NaN
2848,NaN,NaN
2849,NaN,NaN
2918,NaN,NaN
2919,NaN,NaN
2920,NaN,NaN
2922,NaN,NaN
2923,NaN,NaN
2988,NaN,NaN


In [21]:
def extract(text):
    match = re.search(r'\((\d*\.?\d+-?\d*\.?\d+)', text)
    if match:
        value = match.group(1)
        if '-' in value:
            l = [float(num) for num in value.split('-')]
            return np.mean(l)
        else:
            return float(value)
    else:
        return np.nan


In [22]:
all_nan['area']=all_nan['area'].astype(str).apply(extract)

/var/folders/9h/szcjr1mj4c51q1_y7hpk6bvm0000gn/T/ipykernel_27283/4159112979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_nan['area']=all_nan['area'].astype(str).apply(extract)


In [23]:
all_nan[['area','built_up_area']]

,area,built_up_area
2846,NaN,NaN
2847,NaN,NaN
2848,NaN,NaN
2849,NaN,NaN
2918,NaN,NaN
2919,NaN,NaN
2920,NaN,NaN
2922,NaN,NaN
2923,NaN,NaN
2988,NaN,NaN


In [24]:
all_nan['area']=round(all_nan['area']*10.763)

/var/folders/9h/szcjr1mj4c51q1_y7hpk6bvm0000gn/T/ipykernel_27283/1724892780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_nan['area']=round(all_nan['area']*10.763)


In [25]:
all_nan[['area','built_up_area']]

,area,built_up_area
2846,NaN,NaN
2847,NaN,NaN
2848,NaN,NaN
2849,NaN,NaN
2918,NaN,NaN
2919,NaN,NaN
2920,NaN,NaN
2922,NaN,NaN
2923,NaN,NaN
2988,NaN,NaN


In [26]:
# 1sq yard=9 sq ft


In [27]:
def convert_scale(row):
    if np.isnan(row['area']) or np.isnan(row['built_up_area']):
        return row['built_up_area']
    else:
        if round(row['area']/row['built_up_area']) == 9.0:
            return row['built_up_area'] * 9
        elif round(row['area']/row['built_up_area']) == 11.0:
            return row['built_up_area'] * 10.7
        else:
            return row['built_up_area']

In [28]:
all_nan['built_up_area'] = all_nan.apply(convert_scale,axis=1)

/var/folders/9h/szcjr1mj4c51q1_y7hpk6bvm0000gn/T/ipykernel_27283/2760017470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_nan['built_up_area'] = all_nan.apply(convert_scale,axis=1)


In [29]:
all_index=all_nan.index

In [30]:
#updating the data frame
common_indices = df.index.intersection(all_nan.index)
df.loc[common_indices] = all_nan.loc[common_indices]

In [31]:
df.loc[3022]

society                                                 bptp visionnaire
price                                                               5.25
area_persqft                                                     20115.0
areaWithType                                 Plot area 290(242.48 sq.m.)
bedRoom                                                              5.0
bathroom                                                             4.0
balcony                                                               3+
additionalRoom                                              Servant Room
floorNum                                                             3.0
facing                                                        North-East
agePossession                                            0 to 1 Year Old
nearbyLocations        ['Good Earth City Center 2', 'Kunskapsskolan I...
description            Do you wish to buy an independent house in sec...
furnishDetails         ['1 Wardrobe', '1 Fan', '1 E

In [32]:
drop_index=df[(df['super_built_up_area'].isnull())&(df['built_up_area'].isnull())&(df['carpet_area'].isnull())&(df['area'].isnull())].index

In [33]:
df.drop(index=drop_index,axis=0,inplace=True)

In [34]:
df2=df[(df['super_built_up_area'].isnull())&(df['built_up_area'].isnull())&(df['carpet_area'].isnull())]

In [35]:
df2['built_up_area']=df2['area']

/var/folders/9h/szcjr1mj4c51q1_y7hpk6bvm0000gn/T/ipykernel_27283/242641773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['built_up_area']=df2['area']


In [36]:
common_indice = df.index.intersection(df2.index)
df.loc[common_indice] = df2.loc[common_indice]

In [37]:
df[(df['super_built_up_area'].isnull())&(df['built_up_area'].isnull())&(df['carpet_area'].isnull())]

,society,price,area_persqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,sector,area,type,super_built_up_area,built_up_area,carpet_area


In [ ]:
# additional room

In [39]:
df['additionalRoom'].value_counts()

additionalRoom
not avilable                                     1777
Servant Room                                      712
Study Room                                        251
Others                                            238
Pooja Room                                        170
Store Room                                        105
Study Room,Servant Room                            99
Pooja Room,Servant Room                            84
Pooja Room,Study Room,Servant Room,Store Room      75
Servant Room,Others                                60
Pooja Room,Study Room,Servant Room                 59
Pooja Room,Study Room,Servant Room,Others          57
Servant Room,Pooja Room                            38
Servant Room,Store Room                            34
Study Room,Others                                  30
Pooja Room,Study Room                              24
Pooja Room,Store Room                              17
Pooja Room,Others                                  17
Servant Room,

In [40]:
df.drop(index=df[df['additionalRoom']==527.432].index,axis=0,inplace=True)

In [45]:
df['additionalRoom']=df['additionalRoom'].str.lower()
# additional room
new_cols = ['study room', 'servant room', 'store room', 'pooja room', 'others']

# Populate the new columns based on the "additionalRoom" column
for col in new_cols:
    df[col] = df['additionalRoom'].str.contains(col).astype(int)

In [46]:
df.sample(5)

,society,price,area_persqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,sector,area,type,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others
3696,international city by sobha phase 2,7.0,11240.0,Plot area 692(578.6 sq.m.),5.0,5.0,3+,servant room,2.0,South,1 to 5 Year Old,"['Dwarka Sector 21 Metro Station', 'Pacific D2...",Sobha international city phase 1\nUpper duplex...,"['1 Exhaust Fan', '5 Geyser', '7 AC', 'No Bed'...","['Feng Shui / Vaastu Compliant', 'Private Gard...",sector 109,6232.0,house,NaN,6228.0,NaN,0,1,0,0,0
2784,ambience creacions,3.2,11500.0,Super Built up area 2781(258.36 sq.m.),3.0,4.0,3+,not avilable,2.0,North-East,Within 6 months,"['Moulsari Avenue', 'Ambience Mall', 'Old Delh...",Right in heart of gurgaon and delhi is just 2k...,NaN,"['Feng Shui / Vaastu Compliant', 'Security / F...",sector 22,2782.609sq-ft,flat,2781.0,NaN,NaN,0,0,0,0,0
1416,bestech park view ananda,1.45,8011.0,Super Built up area 1810(168.15 sq.m.)Built Up...,3.0,3.0,3,others,8.0,North,1 to 5 Year Old,"['Metro Station Kankrola sec 87', 'Sapphire 83...",This 3 bhk apartment is available for sale in ...,"['3 Fan', '1 Exhaust Fan', '2 Geyser', '4 Ligh...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...",sector 81,1810.011sq-ft,flat,1810.0,1650.0,1500.000000,0,0,0,0,1
272,mapsko casa bella,1.25,6377.0,Super Built up area 1960(182.09 sq.m.),3.0,4.0,3+,not avilable,8.0,NaN,By 2023,"['Vatika City Centre Mall', 'Pataudi Road', 'B...",We are the proud owners of this 3 bhk apartmen...,[],NaN,sector 82,1960.169sq-ft,flat,1960.0,NaN,NaN,0,0,0,0,0
605,emaar palm gardens,1.35,12325.0,Super Built up area 1720(159.79 sq.m.)Carpet a...,3.0,3.0,3,servant room,7.0,South-West,1 to 5 Year Old,"[""V'Lante Mall"", 'NH 48 Gurugram', 'iGrow Mont...",A beautiful 3 bhk apartment in sector-83 gurga...,NaN,"['Lift(s)', 'Swimming Pool', 'Park', 'Fitness ...",sector 83,1095.335sq-ft,flat,1720.0,NaN,1095.334464,0,1,0,0,0


In [47]:
def categorize_age_possession(value):
    if pd.isna(value):
        return "Undefined"
    if "0 to 1 Year Old" in value or "Within 6 months" in value or "Within 3 months" in value:
        return "New Property"
    if "1 to 5 Year Old" in value:
        return "Relatively New"
    if "5 to 10 Year Old" in value:
        return "Moderately Old"
    if "10+ Year Old" in value:
        return "Old Property"
    if "Under Construction" in value or "By" in value:
        return "Under Construction"
    try:
        # For entries like 'May 2024'
        int(value.split(" ")[-1])
        return "Under Construction"
    except:
        return "Undefined"

In [48]:
df['agePossession'] = df['agePossession'].apply(categorize_age_possession)

In [49]:
df.sample(5)

,society,price,area_persqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,sector,area,type,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others
3178,NaN,5.5,28424.0,Plot area 215(179.77 sq.m.),1.0,1.0,0,others,1.0,South-West,Old Property,"['Huda city centre metro station', 'Sector 42-...",Ansal built up independent house khander is av...,"['1 Light', 'No AC', 'No Bed', 'No Chimney', '...","['Feng Shui / Vaastu Compliant', 'Visitor Park...",sector 43,1937.0,house,NaN,1935.0,NaN,0,0,0,0,1
1592,experion windchants,3.0,11538.0,Super Built up area 2802(260.31 sq.m.)Built Up...,3.0,3.0,2,others,2.0,North,Relatively New,"['Ansal Plaza', 'Dwaraka Expressway', 'First S...",This 3 bhk apartment is available for sale in ...,"['5 Fan', '1 Exhaust Fan', '5 Geyser', '18 Lig...","['Feng Shui / Vaastu Compliant', 'Security / F...",sector 112,2600.104sq-ft,flat,2802.0,2600.0,2200.0,0,0,0,0,1
665,cghs airport apartment,1.65,7857.0,Super Built up area 2100(195.1 sq.m.)Carpet ar...,3.0,3.0,3,others,1.0,North-East,Old Property,"['Rajiv Chowk Mosque', 'Standard chartered ATM...",This property located in sector 47 gurgaon. 3 ...,"['5 Fan', '1 Exhaust Fan', '4 Geyser', '10 Lig...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...",sector 47,2100.038sq-ft,flat,2100.0,NaN,1720.0,0,0,0,0,1
386,orchid petals,2.55,12318.0,Super Built up area 2070(192.31 sq.m.)Built Up...,3.0,4.0,3,study room,8.0,East,Relatively New,"['Sapphire Mall', 'Omaxe City Centre', 'BigBaz...","Park pool facing sunny flat full cheque ,this ...","['8 Fan', '1 Exhaust Fan', '3 Geyser', '8 Ligh...","['Feng Shui / Vaastu Compliant', 'Security / F...",sector 49,2070.141sq-ft,flat,2070.0,2060.0,1760.0,1,0,0,0,0
3926,bptp astaire gardens,1.04,7172.0,Carpet area: 1450 (134.71 sq.m.),3.0,3.0,3+,others,2.0,North-East,Relatively New,"['Sector 54 Chowk Metro Station', 'Airia Mall'...",On the lookout for a spacious house in gurgaon...,"['3 Wardrobe', '5 Fan', '1 Exhaust Fan', '3 Ge...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...",sector 70a,(135 sq.m.) Carpet Area,house,NaN,NaN,1450.0,0,0,0,0,1


In [50]:
df.drop(columns=['areaWithType','nearbyLocations','description'],inplace=True)

In [51]:
df.sample(5)

,society,price,area_persqft,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,furnishDetails,features,sector,area,type,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others
3538,sobha city,10.5,14323.0,5.0,4.0,3+,not avilable,1.0,East,Undefined,NaN,NaN,sector 108,(681 sq.m.) Built-up Area,house,NaN,7331.0,NaN,0,0,0,0,0
1933,suncity heights,3.85,16445.0,3.0,3.0,3+,servant room,8.0,NaN,Old Property,"['3 Wardrobe', '3 Fan', '2 Geyser', '4 Light',...","['Power Back-up', 'Intercom Facility', 'Lift(s...",sector 54,2341.137sq-ft,flat,2341.0,NaN,NaN,0,1,0,0,0
2639,the roselia 2,PriceonRequest,NaN,2.0,2.0,2,not avilable,14.0,NaN,Undefined,NaN,NaN,sector 95,nansq-ft,flat,NaN,NaN,593.0,0,0,0,0,0
746,tulip white,0.9,6000.0,3.0,2.0,3+,not avilable,1.0,West,Relatively New,"['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '2 Ge...","['Security / Fire Alarm', 'Power Back-up', 'Fe...",sector 69,1500.0sq-ft,flat,1500.0,1326.0,1050.0,0,0,0,0,0
930,dlf the ultima,2.3,14375.0,3.0,4.0,3+,servant room,18.0,North-East,Relatively New,"['3 Wardrobe', '3 Fan', '1 Fridge', '3 Geyser'...","['Centrally Air Conditioned', 'Water purifier'...",sector 81,1600.0sq-ft,flat,2132.0,NaN,1600.0,0,1,0,0,0


In [ ]:
df.to_csv('gurgaon_feature engeenerring 1 approach 2.csv',index=False)